# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-03-13 09:58:14] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38559, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=88007367, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_time_cost=False, enab

[2025-03-13 09:58:35 TP0] Init torch distributed begin.
[2025-03-13 09:58:35 TP0] Init torch distributed ends. mem usage=1.76 GB
[2025-03-13 09:58:35 TP0] Load weight begin. avail mem=56.40 GB


[2025-03-13 09:58:35 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-13 09:58:35 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.26it/s]


...

[2025-03-13 09:58:40 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768
[2025-03-13 09:58:40] INFO:     Started server process [3886129]
[2025-03-13 09:58:40] INFO:     Waiting for application startup.
[2025-03-13 09:58:40] INFO:     Application startup complete.
[2025-03-13 09:58:40] INFO:     Uvicorn running on http://0.0.0.0:38559 (Press CTRL+C to quit)


[2025-03-13 09:58:41] INFO:     127.0.0.1:60270 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-13 09:58:41] INFO:     127.0.0.1:60284 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-13 09:58:41 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:58:44] INFO:     127.0.0.1:60292 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:44] The server is fired up and ready to roll!


Server started on http://localhost:38559


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-03-13 09:58:46] INFO:     127.0.0.1:53418 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-03-13 09:58:46 TP0] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:58:46] INFO:     127.0.0.1:53432 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-03-13 09:58:46 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:46 TP0] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 6.17, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:47] INFO:     127.0.0.1:53434 - "POST /generate HTTP/1.1" 200 OK


[2025-03-13 09:58:47 TP0] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 71, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:47 TP0] Decode batch. #running-req: 1, #token: 102, token usage: 0.00, gen throughput (token/s): 59.78, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:47] INFO:     127.0.0.1:53440 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-03-13 09:58:47 TP0] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:47] INFO:     127.0.0.1:53442 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:47 TP0] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:58:48] INFO:     127.0.0.1:53454 - "POST /generate HTTP/1.1" 200 OK


[2025-03-13 09:58:48 TP0] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:48 TP0] Decode batch. #running-req: 1, #token: 54, token usage: 0.00, gen throughput (token/s): 50.80, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:48] INFO:     127.0.0.1:53466 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-03-13 09:58:48 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:48 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-03-13 09:58:48 TP0] Decode batch. #running-req: 2, #token: 68, token usage: 0.00, gen throughput (token/s): 115.63, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:48 TP0] Decode batch. #running-req: 2, #token: 148, token usage: 0.01, gen throughput (token/s): 212.01, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:49 TP0] Decode batch. #running-req: 2, #token: 228, token usage: 0.01, gen throughput (token/s): 212.72, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:49 TP0] Decode batch. #running-req: 2, #token: 308, token usage: 0.02, gen throughput (token/s): 216.86, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:50 TP0] Decode batch. #running-req: 2, #token: 388, token usage: 0.02, gen throughput (token/s): 205.72, largest-len: 0, #queue-req: 0, 
[2025-03-13 09:58:50] INFO:     127.0.0.1:53480 - "POST /generate HTTP/1.1" 200 OK


[2025-03-13 09:58:50] INFO:     127.0.0.1:53478 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:50 TP0] Prefill batch. #new-seq: 1, #new-token: 401, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:50 TP0] Decode batch. #running-req: 1, #token: 445, token usage: 0.02, gen throughput (token/s): 120.90, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:50 TP0] Decode batch. #running-req: 1, #token: 485, token usage: 0.02, gen throughput (token/s): 113.38, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:51 TP0] Decode batch. #running-req: 1, #token: 525, token usage: 0.03, gen throughput (token/s): 111.94, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:51 TP0] Decode batch. #running-req: 1, #token: 565, token usage: 0.03, gen throughput (token/s): 106.67, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:51] INFO:     127.0.0.1:53488 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-03-13 09:58:51 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:58:52 TP0] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 36.24, largest-len: 0, #queue-req: 0, 
[2025-03-13 09:58:52] INFO:     127.0.0.1:53504 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-03-13 09:58:53 TP0] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:58:53 TP0] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, gen throughput (token/s): 42.65, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:53 TP0] Decode batch. #running-req: 1, #token: 118, token usage: 0.01, gen throughput (token/s): 101.06, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:58:54 TP0] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, gen throughput (token/s): 92.91, largest-len: 0, #queue-req: 0, 
[2025-03-13 09:58:54] INFO:     127.0.0.1:53518 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:54] INFO:     127.0.0.1:53522 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, token usage: 0.00, #running-req: 1, #queue-req: 0, 


 33%|███▎      | 1/3 [00:00<00:00,  8.06it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 12.76it/s]

100%|██████████| 3/3 [00:00<00:00, 21.12it/s]

[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, token usage: 0.00, #running-req: 2, #queue-req: 0, 
[2025-03-13 09:58:54] INFO:     127.0.0.1:53536 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:54] INFO:     127.0.0.1:53550 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:54] INFO:     127.0.0.1:53532 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-03-13 09:58:54] INFO:     127.0.0.1:53566 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 


The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-03-13 09:58:54 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-13 09:58:54 TP0] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 113.52, largest-len: 0, #queue-req: 0, 
[2025-03-13 09:58:54] INFO:     127.0.0.1:53578 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/references/supported_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-03-13 09:59:06] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37998, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=1065434723, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_time_cost=

[2025-03-13 09:59:27 TP0] Overlap scheduler is disabled for multimodal models.


[2025-03-13 09:59:27 TP0] Automatically reduce --mem-fraction-static to 0.836 because this is a multimodal model.
[2025-03-13 09:59:27 TP0] Init torch distributed begin.


[2025-03-13 09:59:28 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-13 09:59:28 TP0] Load weight begin. avail mem=78.30 GB
[2025-03-13 09:59:29 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-13 09:59:29 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.34it/s]

[2025-03-13 09:59:33 TP0] Load weight end. type=Qwen2_5_VLForConditionalGeneration, dtype=torch.bfloat16, avail mem=38.16 GB, mem usage=40.14 GB.
[2025-03-13 09:59:33 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-03-13 09:59:33 TP0] Memory pool end. avail mem=36.79 GB


[2025-03-13 09:59:34 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=128000
[2025-03-13 09:59:35] INFO:     Started server process [3889022]
[2025-03-13 09:59:35] INFO:     Waiting for application startup.
[2025-03-13 09:59:35] INFO:     Application startup complete.
[2025-03-13 09:59:35] INFO:     Uvicorn running on http://0.0.0.0:37998 (Press CTRL+C to quit)


[2025-03-13 09:59:36] INFO:     127.0.0.1:54694 - "GET /v1/models HTTP/1.1" 200 OK
[2025-03-13 09:59:36] INFO:     127.0.0.1:54696 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-13 09:59:36 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:59:39] INFO:     127.0.0.1:54700 - "POST /generate HTTP/1.1" 200 OK
[2025-03-13 09:59:39] The server is fired up and ready to roll!


Server started on http://localhost:37998


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-03-13 09:59:41] INFO:     127.0.0.1:54702 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-03-13 09:59:48 TP0] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-13 09:59:50 TP0] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, gen throughput (token/s): 2.56, largest-len: 0, #queue-req: 0, 


[2025-03-13 09:59:50] INFO:     127.0.0.1:57430 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-03-13 09:59:51] Child process unexpectedly failed with an exit code 9. pid=3889323
[2025-03-13 09:59:51] Child process unexpectedly failed with an exit code 9. pid=3889164
[2025-03-13 09:59:51] Child process unexpectedly failed with an exit code 9. pid=3891678
